Here are my basic goals and ideas for this project. I want to analyze LoL data from the last 11 years to attempt to find which conditions are most important for a team's victory in the main tournament. 

Potential Factors to explore in no particular order:
  1) Strongest champs per position
  2) Biggest position Contribution per game
    (ratio between 1 and 2 might be something?) 
  3) Strongest Player per team
  4) Tournament Winning teams

data culling:
  only look at main event games
  make a df['game_id'] column


ML model:
  use model to run numbers and assign weights to key features and predict wins 
  and losses
    numbers = [TBD] list of what factors seem important.



In [1]:
import pandas as pd
import numpy as np
import re

In [12]:
champ_df = pd.read_csv('LoL/champion_stats.csv')
player_df = pd.read_csv('LoL/player_stats.csv')
match_df = pd.read_csv('LoL/match_stats.csv')

In [13]:
champ_df.shape


(1345, 24)

# DATA WRANGLING

In [40]:
# Take a stab at a broad-spec wrangle function

def wrangle(df):

  # looking at null sums for the three tables lets me know that columns either have 0 null values or greater than 50 
  # the match_df has several annoying columns that I can get rid of using regex without messing up the other tables
  cutoff = 50
  p = "(ban|pick)"
  pa = "(kill_death_assist_ratio|win_rate)"
  drop_cols = []
  
  df.drop(columns=['event'], inplace=True)

  for col in df.columns:
    if re.match(p, col) != None:
      drop_cols.append(col)
    elif re.fullmatch(pa, col) != None:
      drop_cols.append(col)
    elif df[col].isnull().sum() > cutoff:
      drop_cols.append(col)

  df.drop(columns=drop_cols, inplace=True)

  return df


In [41]:
cdf = champ_df.loc[champ_df['event'] == 'Main']
mdf = match_df.loc[match_df['event'] == 'Main']
pdf = player_df.loc[player_df['event'] == 'Main']

In [42]:
cdf = wrangle(cdf);
mdf = wrangle(mdf);
pdf = wrangle(pdf);

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [17]:
cdf.shape

(867, 15)

Note: to save time, run only the above cells, then skip to next section.

# Matches

In [ ]:
match_df.isnull().sum()

season                 0
date                   0
event                  0
patch                122
blue_team              0
red_team               0
winner                 0
ban_1_blue_team        0
ban_2_blue_team        0
ban_3_blue_team        0
ban_4_blue_team      350
ban_5_blue_team      352
ban_1_red_team         0
ban_2_red_team         0
ban_3_red_team         0
ban_4_red_team       350
ban_5_red_team       350
pick_1_blue_team       0
pick_2_blue_team       0
pick_3_blue_team       0
pick_4_blue_team       0
pick_5_blue_team       0
pick_1_red_team        0
pick_2_red_team        0
pick_3_red_team        0
pick_4_red_team        0
pick_5_red_team        0
top_blue_team          0
jungler_blue_team      0
mid_blue_team          0
adc_blue_team          0
support_blue_team      0
top_red_team           0
jungler_red_team       0
mid_red_team           0
adc_red_team           0
support_red_team       0
dtype: int64

In [ ]:
# I'm only interested in the main tournament
mdf = match_df.loc[match_df['event'] == 'Main']

In [ ]:
mdf.shape

(823, 37)

In [ ]:
wrangle(mdf);
mdf.shape

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(823, 16)

In [ ]:
mdf.columns

Index(['season', 'date', 'event', 'blue_team', 'red_team', 'winner',
       'top_blue_team', 'jungler_blue_team', 'mid_blue_team', 'adc_blue_team',
       'support_blue_team', 'top_red_team', 'jungler_red_team', 'mid_red_team',
       'adc_red_team', 'support_red_team'],
      dtype='object')

In [ ]:
mdf.head()

,season,date,event,blue_team,red_team,winner,top_blue_team,jungler_blue_team,mid_blue_team,adc_blue_team,support_blue_team,top_red_team,jungler_red_team,mid_red_team,adc_red_team,support_red_team
0,1,2011-06-20,Main,Against_All_authority,Fnatic,Fnatic,MoMa,Linak,sOAZ,YellOwStaR,Kujaa,Shushei,Cyanide,xPeke,Lamia,Mellisan
1,1,2011-06-20,Main,Fnatic,Against_All_authority,Against_All_authority,Shushei,Cyanide,xPeke,Lamia,Mellisan,sOAZ,Linak,MoMa,YellOwStaR,Kujaa
2,1,2011-06-20,Main,Against_All_authority,TSM,Against_All_authority,YellOwStaR,Linak,MoMa,sOAZ,Kujaa,Chaox,TheOddOne,Reginald,TheRainMan,Xpecial
3,1,2011-06-20,Main,Epik_Gamer,TSM,TSM,Dyrus,Dan Dinh,Salce,Westrice,Doublelift,TheRainMan,TheOddOne,Reginald,Chaox,Xpecial
4,1,2011-06-20,Main,TSM,Epik_Gamer,TSM,TheRainMan,TheOddOne,Reginald,Chaox,Xpecial,Westrice,Dan Dinh,Salce,Dyrus,Doublelift


In [ ]:
# according to the dataset the cup champion is not always the one who wins the most games, but I'll still be using that metric for my base
# seasons 2 and 11 list the runner up instead of the champion.

season = 0

while season < mdf['season'].nunique():
  season += 1
  tmdf = mdf[mdf['season'] == season]
  print(tmdf['winner'].value_counts().index[0])

Fnatic
Azubu_Frost
SK_Telecom_T1
Samsung_White
SK_Telecom_T1
SK_Telecom_T1
Samsung_Galaxy
Invictus_Gaming
FunPlus_Phoenix
DWG_KIA
DWG_KIA
DRX


In [ ]:
# I can use the method below to help gague player contribution 
mdf[mdf == 'Impact'].count()

season                0
date                  0
event                 0
blue_team             0
red_team              0
winner                0
top_blue_team        31
jungler_blue_team     0
mid_blue_team         0
adc_blue_team         0
support_blue_team     0
top_red_team         31
jungler_red_team      0
mid_red_team          0
adc_red_team          0
support_red_team      0
dtype: int64

*Champions*

# Champions

In [ ]:
champ_df.shape

(1345, 24)

In [ ]:
champ_df.isnull().sum()

season                           0
event                            0
champion                         0
games_contests                   0
pick_ban_ratio                   0
banned_games                   379
played_games                     0
played_by_number_of_players     52
win                              0
lose                             0
win_rate                        52
kills                           52
deaths                          52
assists                         52
kill_death_assist_ratio         52
creep_score                     52
cs/min                          52
gold                            52
gold/min                        52
damage                         865
damage/min                     865
kill_participation             222
kill_share                     222
gold_share                     220
dtype: int64

In [ ]:
# limit df to main event
cdf = champ_df.loc[champ_df['event'] == 'Main']

In [ ]:
wrangle(cdf);
cdf.shape

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(867, 15)

In [ ]:
cdf.columns

Index(['season', 'event', 'champion', 'games_contests', 'played_games',
       'played_by_number_of_players', 'win', 'lose', 'kills', 'deaths',
       'assists', 'creep_score', 'cs/min', 'gold', 'gold/min'],
      dtype='object')

*Player*

# Players

In [ ]:
player_df.shape

(1283, 21)

In [ ]:
player_df.isnull().sum()

season                       0
event                        0
team                         0
player                       0
games_played                 0
wins                         0
loses                        0
win_rate                     0
kills                        0
deaths                       0
assists                      0
kill_death_assist_ratio      0
creep_score                  0
cs/min                       0
gold                         0
gold/min                     0
damage                     874
damage/min                 874
kill_participation         172
kill_share                 172
gold_share                 172
dtype: int64

In [ ]:
pdf = player_df.loc[player_df['event'] == 'Main']

In [ ]:
wrangle(pdf);
pdf.shape

(929, 14)

# Analysis

In [43]:
cdf.columns

Index(['season', 'champion', 'games_contests', 'played_games',
       'played_by_number_of_players', 'win', 'lose', 'kills', 'deaths',
       'assists', 'creep_score', 'cs/min', 'gold', 'gold/min'],
      dtype='object')

In [44]:
mdf.columns

Index(['season', 'date', 'blue_team', 'red_team', 'winner', 'top_blue_team',
       'jungler_blue_team', 'mid_blue_team', 'adc_blue_team',
       'support_blue_team', 'top_red_team', 'jungler_red_team', 'mid_red_team',
       'adc_red_team', 'support_red_team'],
      dtype='object')

In [45]:
pdf.columns

Index(['season', 'team', 'player', 'games_played', 'wins', 'loses', 'kills',
       'deaths', 'assists', 'creep_score', 'cs/min', 'gold', 'gold/min'],
      dtype='object')

In [38]:
# going by season might be too small a dataset to work with effectively

In [39]:
# make sure win loss across data is balanced
# consider re-adding high null count rows/values (sub with mean/median/mode values to estimate) especially helpful with cross validation techniques 

In [11]:
'''
cdf = champion data
mdf = match data
pdf = player data

Ultimately I'm interested in predicting a winner. Should I use the champion or player?
  maybe I should try to work out a new dataframe that links the player to the champion they played? (that might be impossible)

for now let's see what the most important role seems to be, season by season.
  contribution score: TODO
''';

In [58]:
def make_season(df, season):
  return df[df['season'] == season]

In [59]:
m1 = make_season(mdf, 1)

In [60]:
p1 = make_season(pdf, 1)

In [ ]:
# only interested in players from the winning team of each match
# so for each match, take the winning team and figure out what I want from each player (k/d) to start
# return reference to their position
# stats are averages over all games they played that season

In [67]:
p1.head()

,season,team,player,games_played,wins,loses,kills,deaths,assists,creep_score,cs/min,gold,gold/min
0,1,Against_All_authority,Kujaa,12,7,5,0.25,2.58,8.33,13.58,0.34,7.8,198
1,1,Against_All_authority,Linak,12,7,5,1.75,3.58,7.67,113.17,2.86,10.3,259
2,1,Against_All_authority,MoMa,12,7,5,4.17,2.75,5.58,242.25,6.13,12.1,307
3,1,Against_All_authority,sOAZ,12,7,5,4.00,2.92,7.08,214.67,5.43,11.6,293
4,1,Against_All_authority,YellOwStaR,12,7,5,3.75,3.25,5.17,276.33,6.99,13.2,333


In [65]:
m1.head()

,season,date,blue_team,red_team,winner,top_blue_team,jungler_blue_team,mid_blue_team,adc_blue_team,support_blue_team,top_red_team,jungler_red_team,mid_red_team,adc_red_team,support_red_team
0,1,2011-06-20,Against_All_authority,Fnatic,Fnatic,MoMa,Linak,sOAZ,YellOwStaR,Kujaa,Shushei,Cyanide,xPeke,Lamia,Mellisan
1,1,2011-06-20,Fnatic,Against_All_authority,Against_All_authority,Shushei,Cyanide,xPeke,Lamia,Mellisan,sOAZ,Linak,MoMa,YellOwStaR,Kujaa
2,1,2011-06-20,Against_All_authority,TSM,Against_All_authority,YellOwStaR,Linak,MoMa,sOAZ,Kujaa,Chaox,TheOddOne,Reginald,TheRainMan,Xpecial
3,1,2011-06-20,Epik_Gamer,TSM,TSM,Dyrus,Dan Dinh,Salce,Westrice,Doublelift,TheRainMan,TheOddOne,Reginald,Chaox,Xpecial
4,1,2011-06-20,TSM,Epik_Gamer,TSM,TheRainMan,TheOddOne,Reginald,Chaox,Xpecial,Westrice,Dan Dinh,Salce,Dyrus,Doublelift
